# 2. Text Preprocessing
#### Juan Julián Cea Morán
---
As we saw in the EDA section, we are working with text in 3 possible languages: english, spanish and french. We have no indication that the final system can work with more languages, so we will only focus on those three.

This means that we are working with a mulitlingual dataset. To solve this task it will be necessary to apply textual pre-processing adapted to each of the different languages present in the dataset. 

There are several ways to achieve this requierement: for example, it could be done by building regular expressions so all exceptions founded in texts could be captured. Another option would be breaking the data apart by language and use specifict tools for each language.

In this case we are using the second approach, so we have to identify th language of each text and then apply specifict transformation functions.

The first step is to take pandas dataframe from EDA section and use the 'text' column to preprocess each sample.

In [1]:
import pickle
import pandas as pd

docs_df = pickle.load(open("data/docs_df.pkl", 'rb'))

In [6]:
docs_df.head()

,Name,Text,Lang,Category
0,apr-book-0-en.txt,"i read this book because in my town, everyone ...",en,APR
1,apr-book-1-en.txt,recipes appreciated by the family (small and l...,en,APR
2,apr-book-10-en.txt,i say no to ease ..... and not to the author w...,en,APR
3,apr-book-100-en.txt,milady has found a good vein: anita blake. bas...,en,APR
4,apr-book-1000-en.txt,"460 bc, somewhere in greece: ""gentlemen, i dec...",en,APR


Now we have to work with 'Text' column. Then, dump the result as a new column in the dataset.

In [8]:
lang_df = docs_df[['Text', 'Lang']]
lang_df.head()

,Text,Lang
0,"i read this book because in my town, everyone ...",en
1,recipes appreciated by the family (small and l...,en
2,i say no to ease ..... and not to the author w...,en
3,milady has found a good vein: anita blake. bas...,en
4,"460 bc, somewhere in greece: ""gentlemen, i dec...",en


---

### Language identification
There are some python libraries that address this issue. For example: 
* LangID: Developped by Lui Marco, is a standalone Language Identification (LangID) tool bassed on previous researchs by the authors. https://github.com/saffsd/langid.py
* langdetect: Port of Nakatani Shuyo's language-detection library (version from 03/03/2014) to Python. https://github.com/Mimino666/langdetect
* TextBlob: A popular Text Processing package for Python. It includes a module for language detection based on a Google API. Requires Internet conection. https://textblob.readthedocs.io/en/dev/

Let's compare them!

In [78]:
# Let's take text in the 3 languages
en = lang_df.loc[(lang_df['Lang'] == 'en')][0:100]
es = lang_df.loc[(lang_df['Lang'] == 'es')][0:100]
fr = lang_df.loc[(lang_df['Lang'] == 'fr')][0:100]

sample_texts = pd.concat([en, es, fr])

In [123]:
import langid

_langid = [l [0] for l in sample_texts['Text'].apply(langid.classify)]

In [128]:
from langdetect import detect

_langdetect = [l for l in sample_texts['Text'].apply(langdetect.detect)]

In [132]:
from textblob import TextBlob

_textblob = [TextBlob(l).detect_language() for l in sample_texts['Text']]

In [134]:
langid_error =  0
langdetect_error = 0
textblob_error = 0

sample_texts_lang = [s for s in sample_texts['Lang']]

for i in range(0, len(sample_texts_lang)):
    if _langid[i] != sample_texts_lang[i]:
        langid_error += 1
    if _langdetect[i] != sample_texts_lang[i]:
        langdetect_error += 1
    if _textblob[i] != sample_texts_lang[i]:
        textblob_error += 1
        
print('LangID Error:', langid_error)
print('LangDetect Error:', langdetect_error)
print('TextBlob Error:', textblob_error)

LangID Error: 0
LangDetect Error: 1
TextBlob Error: 0


A we can see, we get similar results using any of this packages. We are going to use the first one, langID. Let's try it with the whole dataset.

In [141]:
full_langid = [l [0] for l in lang_df['Text'].apply(langid.classify)]

In [144]:
error = 0

full_langs = [s for s in lang_df['Lang']]

for i in range(0, len(full_langs)):
    if full_langid[i] != full_langs[i]:
        error += 1

print(error)

1656


As it can be seen, this approach can perform well with a little subsample of the original dataset, but it return 1656 miss classifications when using the entire dataset. Due to that, we are going to use a preprocessing function capable of perform well in every language.

---
### Cleaning and tokenize
We have to build a function that takes every text sample and prepares it for being analyzed by our ML algorithms.

In [193]:
import re
import string
import unidecode
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

stopwords = stopwords.words("french") + stopwords.words("english") + stopwords.words("spanish")

def preprocess(text):
    text = unidecode.unidecode(text.strip().lower())
    
    url_regex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = url_regex.sub(' ', text)
    
    html_pattern = re.compile(r'<.?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    text =  html_pattern.sub(' ', text)
    
    chars_regex = re.compile(r'[^\w\s]')
    tweet = chars_regex.sub(' ', text)
    
    number_regex = re.compile(r'[\d]+')
    text = number_regex.sub(' ', text)
    
    text = ' '.join(text.split('\''))
    
    text = word_tokenize(text)
    
    text = [word for word in text if]
    
    text = [word for word in text if word not in stopwords and len(word)>2]
    
    return text

In [194]:
preprocess("I'aafm ___42ababa")

['aafm', '___', 'ababa']

In [168]:
%%time
import pandas as pd

preprocessed = list()
for text in docs_df['Text']:
    preprocessed.append(preprocess(text))
    
docs_df = docs_df.drop(columns=['Preprocessed'])
docs_df.insert(2, 'Preprocessed', preprocessed)

Wall time: 9min 24s


In [169]:
docs_df.head()

,Name,Text,Preprocessed,Lang,Category
0,apr-book-0-en.txt,"i read this book because in my town, everyone ...","[read, book, town, everyone, uses, order, phar...",en,APR
1,apr-book-1-en.txt,recipes appreciated by the family (small and l...,"[recipes, appreciated, family, small, large, r...",en,APR
2,apr-book-10-en.txt,i say no to ease ..... and not to the author w...,"[say, ease, ....., author, even, made, effort,...",en,APR
3,apr-book-100-en.txt,milady has found a good vein: anita blake. bas...,"[milady, found, good, vein, anita, blake, base...",en,APR
4,apr-book-1000-en.txt,"460 bc, somewhere in greece: ""gentlemen, i dec...","[somewhere, greece, gentlemen, decided, visit,...",en,APR


---
### Explore results
Now that we have our texts preprocessed, it's time to explore the data and check if there are irrelevant terms not considered as stop words. This process is important because we want our model to perform well by only "paying attention" to relevant terms in the corpus.

Let's generate TF-IDF of the results extract some insights. For this test, lets separate texts by language. For the moment we don't need to worry about categories.

In [170]:
en_texts = [' '.join(text) for text in docs_df.loc[(docs_df['Lang'] == 'en')]['Preprocessed']]
es_texts = [' '.join(text) for text in docs_df.loc[(docs_df['Lang'] == 'es')]['Preprocessed']]
fr_texts = [' '.join(text) for text in docs_df.loc[(docs_df['Lang'] == 'fr')]['Preprocessed']]

# total = [' '.join(text) for text in docs_df.loc[(docs_df['Lang'] == 'es')]['Preprocessed'][0:10000]]

In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer 

# English
en_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
en_tfidf = en_tfidf_vectorizer.fit_transform(en_texts)
en_tfidf_feature_names = en_tfidf_vectorizer.get_feature_names()

# Spanish
es_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
es_tfidf = es_tfidf_vectorizer.fit_transform(es_texts)
es_tfidf_feature_names = es_tfidf_vectorizer.get_feature_names()

# French
fr_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
fr_tfidf = fr_tfidf_vectorizer.fit_transform(fr_texts)
fr_tfidf_feature_names = fr_tfidf_vectorizer.get_feature_names()

# fr_tfidf_vectorizer = TfidfVectorizer(use_idf=True)
# fr_tfidf = fr_tfidf_vectorizer.fit_transform(total)
# fr_tfidf_feature_names = fr_tfidf_vectorizer.get_feature_names()


In [173]:
en_tfidf_feature_names

['__',
 '____',
 '__________',
 '_____________',
 '____data',
 '____it',
 '____surrounded',
 '__forcetoc__location',
 '__init__',
 '__main__',
 '__name__',
 '__notoc__',
 '__notoc__ar',
 '__notoc__les',
 '__toc__',
 '__toc__descriptionginkgos',
 '__toc__etymologythe',
 '__toc__etymologywas',
 '__toc__historywith',
 '__toc__overviewayurveda',
 '__toc__taxonomythe',
 '__traverse__',
 '__venezia',
 '_adidas_snake_white',
 '_adidasonwhite',
 '_adidasstripesonwhite',
 '_adidasstripeswhite',
 '_adidaswhite',
 '_agency',
 '_alpha',
 '_analekta',
 '_article',
 '_asnl',
 '_asnl_ext',
 '_black_background',
 '_black_stripes',
 '_blackborder',
 '_blackbrown',
 '_blackshoulders',
 '_blackshoulders_blacksides',
 '_blackstripes',
 '_blueline',
 '_blueshoulders',
 '_bluestripes',
 '_blueupper',
 '_borderonblack',
 '_borderonwhite',
 '_bush_john_paul_ii_funeral',
 '_caption',
 '_chi',
 '_collarblue',
 '_collaronwhite',
 '_collarred',
 '_color',
 '_country',
 '_dp',
 '_dt',
 '_dv',
 '_dzheims',
 '_e',
 

---
Now we have to export data so we can work with it in the next sections.

In [125]:
preproc_df = docs_df.drop(columns=["Name", "Text", "Lang"])

In [128]:
import pickle

output = open('data/preproc_df.pkl', 'wb')
pickle.dump (preproc_df, output)
output.close()